# ERDDAP Explorer

In [1]:
import warnings


warnings.simplefilter('ignore')

In [2]:
from erddapy import ERDDAP


# server = 'https://erddap-uncabled.oceanobservatories.org/uncabled/erddap'
server = 'http://erddap.secoora.org/erddap'
e = ERDDAP(server=server, protocol='tabledap')

In [3]:
import pendulum


initial_standard_name = 'sea_surface_height_above_sea_level'

cdm_data_type = 'TimeSeries'

max_time = pendulum.now(tz='UTC')
min_time = max_time.subtract(weeks=2)

In [4]:
import pandas as pd


url = f'{server}/categorize/standard_name/index.csv'

df = pd.read_csv(url, skiprows=[1, 2])

# variables = df['Category'].values.tolist()
variables = [
    'relative_humidity',
    'river_discharge',
    'sea_surface_height_above_sea_level',
    'sea_surface_wave_significant_height',
    'sea_surface_wind_wave_significant_height',
    'sea_water_ph_reported_on_total_scale',
    'sea_water_practical_salinity',
    'sea_water_pressure',
    'sea_water_temperature',
    'solar_radiation',
    'water_surface_height_above_reference_datum',
    'wind_speed',
]

In [6]:
import ipywidgets


dpdown = ipywidgets.Dropdown(
    options=variables,
    value=initial_standard_name
)

In [7]:
def point(dataset, lon, lat):
    geojsonFeature = {
        'type': 'Feature',
        'properties': {
            'datasetID': dataset,
        },
        'geometry': {
            'type': 'Point',
            'coordinates': [lon, lat]
        }
    };
    geojsonFeature['properties']['style'] = {'color': 'Grey'}
    return geojsonFeature

In [8]:
def adv_search(e, standard_name, min_time, max_time):
    try:
        search_url = e.get_search_url(
            response='csv',
            cdm_data_type='timeseries',
            items_per_page=100000,
            standard_name=standard_name,
            min_time=min_time,
            max_time=max_time
        )
        df = pd.read_csv(search_url)
    except Exception:
        df = []
        if len(var) > 14:
            v = '{}...'.format(standard_name[:15])
        else:
            v = standard_name
        figure.title = f'No {v} found in this time range. Pick another variable.'
        figure.marks[0].y = 0.0 * figure.marks[0].y
    return df

In [9]:
import io
import requests


def all_positions(e, cdm_data_type, min_time, max_time):
    url = (
        f'{e.server}/tabledap/allDatasets.csv?'
        f'datasetID,minLongitude,minLatitude'
        f'&cdm_data_type="{cdm_data_type}"'
        f'&minTime<={max_time}'
        f'&maxTime>={min_time}'
    )
    data = io.BytesIO(requests.get(url).content)
    df = pd.read_csv(data, skiprows=[1])
    return df

In [10]:
def stdname2geojson(e, standard_name, min_time, max_time):
    search = adv_search(e, standard_name, min_time, max_time)
    
    if isinstance(search, pd.DataFrame):
        datasets = search['Dataset ID'].values
        positions = all_positions(e, cdm_data_type, min_time, max_time)
        data = positions[positions['datasetID'].isin(search['Dataset ID'])]

        geojson = {
            'features': [
                point(row[1], row[2], row[3]) for row in data.itertuples()
            ]
        }
    else:
        geojson = {
            'features': []
        }
        datasets = []
    return geojson, datasets

In [11]:
def click_handler(event=None, id=None, properties=None):
    datasetID = properties['datasetID']

    kwargs = {
        'time>=': min_time,
        'time<=': max_time
    }

    df, var = get_data(datasetID, dpdown.value, kwargs)

    figure.marks[0].x = df.index
    figure.marks[0].y = df[var]
    figure.title = f'{properties["datasetID"]}'

In [12]:
from ipyleaflet import GeoJSON


def update_dpdown(change):
    standard_name = change['new']
    data, datasets = stdname2geojson(e, standard_name, min_time, max_time)
    feature_layer = GeoJSON(data=data)
    feature_layer.on_click(click_handler)
    m.layers = [m.layers[0], feature_layer]

In [13]:
dpdown.observe(update_dpdown, names=['value'])

In [14]:
def get_data(dataset_id, standard_name, kwargs):
    var = e.get_var_by_attr(
        dataset_id=dataset_id,
        standard_name=lambda v: str(v).lower() == standard_name)[0]
    e.dataset_id = dataset_id
    e.variables = ['time', var]
    e.constraints = kwargs
    df = e.to_pandas(index_col='time', parse_dates=True, skiprows=[1])
    return df, var

In [15]:
from ipyleaflet import Map, basemaps



m = Map(
    basemap=basemaps.Esri.NatGeoWorldMap,
    center=[29.67, -80.82],
    zoom=5,
)


data, datasets = stdname2geojson(e, initial_standard_name, min_time, max_time)
feature_layer = GeoJSON(data=data)
feature_layer.on_click(click_handler)
m.layers = [m.layers[0], feature_layer]

In [17]:
import bqplot


dt_x = bqplot.DateScale()
sc_y = bqplot.LinearScale()

initial_dataset = datasets[0]
kwargs = {'time>=': min_time, 'time<=': max_time}

df, var = get_data(initial_dataset, initial_standard_name, kwargs)

time_series = bqplot.Lines(x=df.index, y=df[var], scales={'x': dt_x, 'y': sc_y})

ax_x = bqplot.Axis(scale=dt_x, label='Time')
ax_y = bqplot.Axis(scale=sc_y, orientation='vertical')

figure = bqplot.Figure(marks=[time_series], axes=[ax_x, ax_y])
figure.title = f'{initial_dataset}'
figure.layout.height = '300px'
figure.layout.width = '800px'

In [18]:
ipywidgets.VBox([dpdown, m, figure])